In [1]:
%matplotlib notebook
%config InlineBackend.print_figure_kwargs = {'bbox_inches':None}

from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from os import path
import numpy as np
import cv2 as cv
import trimesh
import os

# Our local modules
import src.calibrate as clb
import src.util as util

# Overview

We use our estimates of camera intrinsics and extrinsics, as well as endoscope markers rigidbody template, to produce a 3D render of the phantom as the endoscope would see it. We then compare the result to the actual image from the endoscope.

> **Note that we only need to run this code once in the beginning of the experiment.**

# Check that we're in the correct folder

For the code in this notebook to work correctly, the Jupyter server must be started from the `endoscope-calibration` folder. Then you put all of your data in the `./endoscope-calibration/data/`. 

In [2]:
# Check that we're in the right directory
base_dir_name = 'endoscope-calibration'
cwd = os.getcwd()
if not cwd.endswith(base_dir_name):
    print('[WARNING] Your base directory is not \'{}\' - are you running '
          'Jupyter in the correct server?'.format(base_dir_name))
    
data_dir = path.join(cwd, 'data')

# Load endoscope marker meshes from the phantom CT scan

In [3]:
endo_phantom_dir = path.join(data_dir, 'endo_phantom')
endo_phantom_markers_file = path.join(endo_phantom_dir, 'mesh1_endoscope_markers.stl')
endo_phantom_markers_joint = trimesh.load(endo_phantom_markers_file)

# Load our camera intrinsics/extrinsics estimates

In [4]:
intrinsics_file = path.join(data_dir, 'refined_intrinsics.json')
cam_matrix, dist_coeffs, width, height = util.load_intrinsics(intrinsics_file)
extrinsics_file = path.join(data_dir, 'extrinsics.json')
T_rigid_to_cam, endoscope_markers = util.load_extrinsics(extrinsics_file)

new_cam_matrix, roi = cv.getOptimalNewCameraMatrix(cam_matrix, dist_coeffs,
                                                   (width, height), 1, (width, height))

# Find transform from world to our calibration reference frame

In [5]:
phantom_centroids = util.extract_marker_mesh_centroids(endo_phantom_markers_joint)
phantom_points_3d = np.array(phantom_centroids).T

T_world_to_rigid = clb.calc_rigid_body_transform(phantom_points_3d, endoscope_markers)

# Find transform from world to camera frame and visualise it

In [18]:
T_world_to_cam = T_rigid_to_cam @ T_world_to_rigid
T_cam_to_world = np.linalg.inv(T_world_to_cam)

T_rigid_to_world = clb.calc_rigid_body_transform(endoscope_markers, phantom_points_3d)
test_points = util.apply_transform(endoscope_markers, T_rigid_to_world)

print(np.round(T_rigid_to_world))
print(np.round(np.linalg.inv(T_world_to_rigid)))

_, ax = util.prepare_3d_plot(title='Camera position with respect to phantom CT scan frame')
util.draw_3d_points(ax, endoscope_markers, colour='blue')
util.draw_3d_points(ax, phantom_points_3d, colour='purple')
util.draw_3d_points(ax, test_points, colour='green')
# util.draw_3d_camera(ax, width, height, new_cam_matrix, T_cam_to_world, z=40)

[[   0.    1.    0.  -92.]
 [  -1.    0.   -0. -121.]
 [  -0.   -0.    1. -113.]
 [   0.    0.    0.    1.]]


AttributeError: module 'numpy' has no attribute 'inv'